## Underscore in python

There is no strict enforcement, but there are some conventions. An attribute that starts with
an underscore is meant to be private to that object, and we expect that no external agent
calls it (but again, there is nothing preventing this).

Objects should only expose those attributes and methods that are relevant
to an external caller object, namely, entailing its interface. Everything that
is not strictly part of an object's interface should be kept prefixed with a
single underscore.

In [1]:
# podemos acceder a todos los atributos de la clase
class Atributos:
    def __init__(self, nombre):
        self.nombre = nombre
        self._edad = 24

a = Atributos("Andrés")
print(a.nombre)
a._edad

Andrés


24

In [12]:
# simulamos un atributo privado
class Atributos:
    def __init__(self, nombre):
        self.nombre = nombre
        self.__edad = 24
    
    def edad(self):
        return self.__edad

a = Atributos("Andrés")
print(a.nombre)
print(a.edad())

Andrés
24


In [14]:
# si lo intentamos acceder directamente...
a.__edad

AttributeError: 'Atributos' object has no attribute '__edad'

Some developers use this method to hide some attributes, thinking, like in this example,
that timeout is now private and that no other object can modify it. Now, take a look at
the exception that is raised when trying to access `__timeout`. It's AttributeError, saying
that it doesn't exist. It doesn't say something like "this is private" or "this can't
be accessed" and so on. It says it does not exist. This should give us a clue that, in fact,
something different is happening and that this behavior is instead just a side effect, but not
the real effect we want.

**Double underscores are a non-Pythonic approach. If you need to define attributes as
private, use a single underscore, and respect the Pythonic convention that it is a private
attribute.**

In [15]:
# entonces lo que en realidad pasa es que se crea con un nombre diferente
a._Atributos__edad

24

**
Objects should only expose those attributes and methods that are relevant
to an external caller object, namely, entailing its interface. Everything that
is not strictly part of an object's interface should be kept prefixed with a
single underscore.**

## Propierties

You might find that properties are a good way to achieve command and query separation . Command and query separation state that a method of an object should either answer to something or do something, but not both. If a method of an object is doing something and at the same time it returns a status answering a question of how that operation went, then it's doing more than one thing, clearly violating the principle that functions should do one thing, and one thing only.


**Methods should do one thing only. If you have to run an action and then
check for the status, so that in separate methods that are called by
different statements.**



In [6]:
class Celsius:
    def __init__(self, temperature = 0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

In [7]:
man = Celsius()
man.temperature = 37
print(man.temperature)
man.to_fahrenheit()

37


98.60000000000001

uggested that temperatures cannot go below -273 degree Celsius (students of thermodynamics might argue that it's actually -273.15), also called the absolute zero. He further asked us to implement this value constraint. 

An obvious solution to the above constraint will be to hide the attribute temperature (make it private) and define new getter and setter interfaces to manipulate it.

In [9]:
# _temperature. An underscore (_) at the beginning is 
# used to denote private variables in Python.
class Celsius:
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    def get_temperature(self):
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value

In [10]:
c = Celsius(-277)

ValueError: Temperature below -273 is not possible

In [11]:
c = Celsius(37)
c.get_temperature()

37

In [13]:
c.set_temperature(10)

In [14]:
c.set_temperature(-300)

ValueError: Temperature below -273 is not possible

Please note that private variables don't exist in Python. There are simply norms to be followed. The language itself don't apply any restrictions.

In [15]:
c._temperature = -300
c.get_temperature()

-300

The big problem with the above update is that, all the clients who implemented our previous class in their program have to modify their code from obj.temperature to obj.get_temperature() and all assignments like obj.temperature = val to obj.set_temperature(val).

### The Power of @property

The pythonic way to deal with the above problem is to use property. Here is how we could have achieved it.

With properties, we can avoid this kind of confusion. The `@property` decorator is the query that will answer to something, and the `@<property_name>.setter` is the command that will do something.

In [16]:
class Celsius:
    def __init__(self, temperature = 0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    def get_temperature(self):
        print("Getting value")
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

    # makes a property object temperature
    temperature = property(get_temperature,set_temperature)

**Any code that retrieves the value of temperature will automatically call get_temperature() instead of a dictionary (__dict__) look-up. Similarly, any code that assigns a value to temperature will automatically call set_temperature()**

In [18]:
# incluso cuando no se llama directamente el set_temperature
# es llamado automáticamente de igual forma con el get
c = Celsius()

Setting value


In [19]:
c.temperature

Getting value


0

In [20]:
c.temperature = 37

Setting value


In [21]:
c.to_fahrenheit()

Getting value


98.60000000000001

the actual temperature value is stored in the private variable `_temperature`. The attribute temperature is a property object which provides interface to this private variable.

In Python, property() is a built-in function that creates and returns a property object. The signature of this function is

    property(fget=None, fset=None, fdel=None, doc=None)

fget is function to get value of the attribute, fset is function to set value of the attribute, fdel is function to delete the attribute and doc is a string (like a comment).

In [22]:
property()

A property object has three methods, getter(), setter(), and deleter() to specify fget, fset and fdel at a later point. This means, the line

se podría haber hecho así también
    # make empty property
    temperature = property()

    # assign fget
    temperature = temperature.getter(get_temperature)
    
    # assign fset
    temperature = temperature.setter(set_temperature)


We can further go on and not define names `get_temperature` and `set_temperature` as they are unnecessary and pollute the class namespace. For this, we reuse the name temperature while defining our getter and setter functions. This is how it can be done.

In [24]:
class Celsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print("Getting value")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

In [25]:
c = Celsius()

In [30]:
c.temperature = 1

Setting value


---